In [5]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import Pmod_IO
import time
IR_PIN = 0
try:
    # Load overlay
    base = BaseOverlay("base.bit")
    
    # Initialize PMODA pin 0 as output
    ir_gpio = Pmod_IO(base.PMODA, 0, 'out')
except Exception as e:
    print(f"Error: {str(e)}")
# NEC Protocol Parameters
CARRIER_FREQ = 38000  # 38 kHz
BIT_TIME = 0.05625  # 562.5 µs (NEC bit duration)
CLOCK_PERIOD = 1 / (2 * CARRIER_FREQ)  # 38 kHz toggle time

# NEC Functions
def send_nec_pulse(duration):
    """Generate a modulated NEC pulse (38 kHz carrier)"""
    end_time = time.time() + duration
    while time.time() < end_time:
        ir_gpio.write(1)  # HIGH
        time.sleep(CLOCK_PERIOD)
        ir_gpio.write(0)  # LOW
        time.sleep(CLOCK_PERIOD)

def send_nec_space(duration):
    """Generate a LOW space (no modulation)"""
    ir_gpio.write(0)  # LOW
    time.sleep(duration)

def send_nec_bit(bit):
    """Send a single NEC bit (1 or 0)"""
    send_nec_pulse(BIT_TIME)  # Always 562.5µs pulse
    if bit == 1:
        send_nec_space(BIT_TIME * 3)  # 1.6875ms space (logic '1')
    else:
        send_nec_space(BIT_TIME)      # 562.5µs space (logic '0')

def send_nec_command(address, command):
    """Send a full NEC IR command (address + command)"""
    # NEC frame: [9ms pulse, 4.5ms space, address, ~address, command, ~command]
    
    # 1. Start pulse (9ms pulse + 4.5ms space)
    send_nec_pulse(0.9)      # 9ms pulse
    send_nec_space(0.45)     # 4.5ms space
    
    # 2. Send address + inverted address
    for bit in range(8):
        send_nec_bit((address >> bit) & 0x1)
    for bit in range(8):
        send_nec_bit(~(address >> bit) & 0x1)
    
    # 3. Send command + inverted command
    for bit in range(8):
        send_nec_bit((command >> bit) & 0x1)
    for bit in range(8):
        send_nec_bit(~(command >> bit) & 0x1)
    
    # 4. Stop bit (562.5µs pulse)
    send_nec_pulse(BIT_TIME)
    ir_gpio.write(0)  # Ensure LED is off

# Example: Send NEC commands
def send_ir_remote_code(button_code):
    """Map button codes to NEC commands (adjust as needed)"""
    nec_codes = {
        'power':(0x00, 0x45),
        '+':(0x00, 0x15),
        '-':(0x00, 0x07),
        '1':(0x00, 0x0C),
        '2':(0x00, 0x18),
        '3':(0x00, 0x5e),
        '4':(0x00, 0x08),
        '5':(0x00, 0x1c),
        '6':(0x00, 0x5a),
        '7':(0x00, 0x42),
        '8':(0x00, 0x52),
        '9':(0x00, 0x4a),
    }
    if button_code in nec_codes:
        addr, cmd = nec_codes[button_code]
        send_nec_command(addr, cmd)
    else:
        print("Unknown command")
while(True):
    number = str(input("Please input number"))
    send_ir_remote_code(number)
    bool_str = str(input("Do you want to send signal again, Y/N"))
    if(bool_str == 'Y'):
        continue
    else:
        break

Please input number5
Do you want to send signal again, Y/NY
Please input number2
Do you want to send signal again, Y/NY
Please input number-
Do you want to send signal again, Y/NN


In [ ]:
help(pynq.lib)